# Advanced Cepstral Analysis

#### Import required libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.fftpack import dct

#### Extract and Visualize MFCCs

##### **Step 1. Load Audio Signal**
The code reads an audio file using `wavfile.read()` and retrieves the sample rate and audio signal data. 
- If the audio signal is stereo (having multiple channels), it is converted to mono by averaging the channels. This is done to simplify further processing, as LPC analysis typically operates on single-channel signals.

##### **Step 2. Preprocess the Audio Signal**
Before extracting MFCCs, the audio signal will be preprocessed:
1. Convert the signal to mono if it is stereo.
2. Normalize the audio signal to have a consistent amplitude.

##### **Step 3. Calculate the Power Spectrum**
The power spectrum will be computed using the Short-Time Fourier Transform (STFT). The STFT will allow the analysis of the frequency content of the audio signal over time.

##### **Step 4. Compute MFCCs**
Using the power spectrum, the MFCCs will be calculated. This involves applying a mel filter bank to the power spectrum, followed by taking the discrete cosine transform (DCT) of the log powers.

##### **Step 5. Visualize MFCCs**
The MFCCs will be visualized in a heatmap format to analyze how these coefficients vary over time.


#### Tips:
- Ако факултетният ви номер е четен използвайте речеви сигнал, а ако е нечетен - музикален сигнал.

In [ ]:
# Load the audio file
sample_rate, signal = wavfile.read('audio_samples_05/FILE_NAME.wav')  # Replace with your file path

# Check if the signal is stereo and convert to mono if needed
if len(signal.shape) > 1:
    signal = signal[:, 0]  # Take only one channel if it's a stereo file

# Normalize audio signal
signal = signal / np.max(np.abs(signal))  # Normalize to the range [-1, 1]

# Define parameters for STFT and MFCC calculation
frame_size = ?   # Set for typical value
hop_size = ?     # Set for typical value
num_mel_filters = 40
num_mfcc = 13

# Calculate the Mel filter bank directly
mel_points = np.linspace(0, 2595 * np.log10(1 + (sample_rate / 2) / 700), num_mel_filters + 2)
hz_points = 700 * (10**(mel_points / 2595) - 1)
bin_points = np.floor((frame_size + 1) * hz_points / sample_rate).astype(int)

mel_filters = np.zeros((num_mel_filters, int(np.floor(frame_size / 2 + 1))))
for m in range(1, num_mel_filters + 1):
    f_m_minus_1 = bin_points[m - 1]
    f_m = bin_points[m]
    f_m_plus_1 = bin_points[m + 1]

    for k in range(f_m_minus_1, f_m):
        mel_filters[m - 1, k] = (k - bin_points[m - 1]) / (f_m - f_m_minus_1)
    for k in range(f_m, f_m_plus_1):
        mel_filters[m - 1, k] = (bin_points[m + 1] - k) / (f_m_plus_1 - f_m)

# Compute the number of frames and pad the signal if necessary
num_frames = int(np.ceil(len(signal) / hop_size))
padded_signal_length = num_frames * hop_size + frame_size
padded_signal = np.zeros(padded_signal_length)
padded_signal[:len(signal)] = signal

# Calculate STFT
stft_matrix = np.array([np.fft.rfft(padded_signal[i * hop_size:i * hop_size + frame_size])
                        for i in range(num_frames)])

# Compute the power spectrum
power_spectrum = np.abs(stft_matrix) ** 2

# Apply the Mel filter bank to the power spectrum
mel_power = np.dot(power_spectrum, mel_filters.T)

# Take the log of the Mel power spectrum
log_mel_power = np.log(mel_power + 1e-12)  # Add epsilon to avoid log(0)

# Compute MFCCs by applying DCT
mfccs = dct(log_mel_power, type=2, axis=1, norm='ortho')[:, :num_mfcc]

# Plot MFCCs
plt.figure(figsize=(12, 6))
plt.imshow(mfccs.T, aspect='auto', origin='lower', cmap='jet', extent=(0, mfccs.shape[0], 0, num_mfcc))
plt.title("Mel-Frequency Cepstral Coefficients (MFCCs)")
plt.xlabel("Frame Index")
plt.ylabel("MFCC Coefficient Index")
plt.colorbar(label='MFCC Value')
plt.tight_layout()
plt.grid(True)
plt.show()

#### Comparing MFCCs of Two Different Audio Samples

##### **Step 1. Load Two Audio Signals**
Load two different audio files. These can be any two sounds. Convert each to mono if necessary and normalize to a common range.

##### **Step 2. Configure Parameters and Prepare Mel Filter Bank**
Define the parameters for Short-Time Fourier Transform (STFT), Mel filter bank, and MFCC calculation, as in the previous exercise. This ensures that both audio samples are processed consistently.

##### **Step 3. Calculate STFT and MFCCs for Each Audio*
For each signal:
1. Perform the STFT to break down the audio into frames.
2. Calculate the power spectrum of each frame.
3. Apply the Mel filter bank and take the logarithm to obtain the Mel spectrogram.
4. Use DCT to extract MFCCs from the log Mel spectrogram.

##### **Step 4. Plot and Compare MFCCs**
Plot the MFCCs of both audio samples side-by-side for comparison. This allows for a visual assessment of their spectral features.

#### Tips:
- Ако факултетният ви номер е четен използвайте музикални сигнали, а ако е нечетен - речеви сигнали.

In [ ]:
# Load two audio files (instrument samples)
file_path1 = 'audio_samples_05/FILE_NAME.wav'  # Replace with your file path
file_path2 = 'audio_samples_05/FILE_NAME.wav'  # Replace with your file path

sample_rate1, signal1 = wavfile.read(file_path1)
sample_rate2, signal2 = wavfile.read(file_path2)

# Ensure both audio signals are mono
if len(signal1.shape) > 1:
    signal1 = signal1[:, 0]
if len(signal2.shape) > 1:
    signal2 = signal2[:, 0]

# Normalize both signals
signal1 = signal1 / np.max(np.abs(signal1))
signal2 = signal2 / np.max(np.abs(signal2))

# Define parameters
frame_size = ?   # Set for typical value
hop_size = ?     # Set for typical value
num_mel_filters = 40
num_mfcc = 13

# Mel filter bank (re-used for both signals)
mel_points = np.linspace(0, 2595 * np.log10(1 + (sample_rate1 / 2) / 700), num_mel_filters + 2)
hz_points = 700 * (10**(mel_points / 2595) - 1)
bin_points = np.floor((frame_size + 1) * hz_points / sample_rate1).astype(int)

mel_filters = np.zeros((num_mel_filters, int(np.floor(frame_size / 2 + 1))))
for m in range(1, num_mel_filters + 1):
    f_m_minus_1 = bin_points[m - 1]
    f_m = bin_points[m]
    f_m_plus_1 = bin_points[m + 1]

    for k in range(f_m_minus_1, f_m):
        mel_filters[m - 1, k] = (k - bin_points[m - 1]) / (f_m - f_m_minus_1)
    for k in range(f_m, f_m_plus_1):
        mel_filters[m - 1, k] = (bin_points[m + 1] - k) / (f_m_plus_1 - f_m)

# Process each signal to calculate MFCCs

# For signal 1
num_frames1 = int(np.ceil(len(signal1) / hop_size))
padded_signal1 = np.zeros(num_frames1 * hop_size + frame_size)
padded_signal1[:len(signal1)] = signal1
stft_matrix1 = np.array([np.fft.rfft(padded_signal1[i * hop_size:i * hop_size + frame_size]) for i in range(num_frames1)])
power_spectrum1 = np.abs(stft_matrix1) ** 2
mel_power1 = np.dot(power_spectrum1, mel_filters.T)
log_mel_power1 = np.log(mel_power1 + 1e-12)
mfccs1 = dct(log_mel_power1, type=2, axis=1, norm='ortho')[:, :num_mfcc]

# For signal 2
num_frames2 = int(np.ceil(len(signal2) / hop_size))
padded_signal2 = np.zeros(num_frames2 * hop_size + frame_size)
padded_signal2[:len(signal2)] = signal2
stft_matrix2 = np.array([np.fft.rfft(padded_signal2[i * hop_size:i * hop_size + frame_size]) for i in range(num_frames2)])
power_spectrum2 = np.abs(stft_matrix2) ** 2
mel_power2 = np.dot(power_spectrum2, mel_filters.T)
log_mel_power2 = np.log(mel_power2 + 1e-12)
mfccs2 = dct(log_mel_power2, type=2, axis=1, norm='ortho')[:, :num_mfcc]

# Plot MFCCs for both signals for comparison
plt.figure(figsize=(14, 8))

# Plot MFCCs for first signal
plt.subplot(2, 1, 1)
plt.imshow(mfccs1.T, aspect='auto', origin='lower', cmap='jet', extent=(0, mfccs1.shape[0], 0, num_mfcc))
plt.title(f"MFCCs for {file_path1}")
plt.xlabel("Frame Index")
plt.ylabel("MFCC Coefficient Index")
plt.colorbar(label='MFCC Value')

# Plot MFCCs for second signal
plt.subplot(2, 1, 2)
plt.imshow(mfccs2.T, aspect='auto', origin='lower', cmap='jet', extent=(0, mfccs2.shape[0], 0, num_mfcc))
plt.title(f"MFCCs for {file_path2}")
plt.xlabel("Frame Index")
plt.ylabel("MFCC Coefficient Index")
plt.colorbar(label='MFCC Value')

plt.tight_layout()
plt.show()

#### Averaging and Comparing MFCC Patterns

##### **Step 1. Load and Normalize the Audio Signals*
Three different audio files will be loaded, converted to mono if necessary, and normalized to ensure consistent comparisons.

##### **Step 2. Define Parameters and Compute Mel Filter Bank**
Define the frame size, hop size, and number of Mel filters to be applied consistently across all samples. The Mel filter bank will also be calculated once and reused for each audio sample to produce the MFCCs.

##### **Step 3. Calculate STFT, Apply Mel Filter Bank, and Compute MFCCs**
Compute the Short-Time Fourier Transform (STFT), apply the Mel filter bank, and calculate MFCCs using the DCT for each audio file.

##### **Step 4. Average MFCCs Across Frames**
For each sample, calculate the mean MFCC values across all frames. This generates a single average MFCC vector per sample, which represents its overall spectral characteristics.

##### **Step 5. Plot Average MFCC Profiles for Each Sample**
Plot the average MFCCs for each sample on the same graph, using different colors, to visually compare the general characteristics of each audio type.

#### Tips:
- Използвайте музикални сигнали.

In [ ]:
# Load three audio files
file_path1 = 'audio_samples_05/FILE_NAME.wav'  # Replace with your file path
file_path2 = 'audio_samples_05/FILE_NAME.wav'  # Replace with your file path
file_path3 = 'audio_samples_05/FILE_NAME.wav'  # Replace with your file path

sample_rate1, signal1 = wavfile.read(file_path1)
sample_rate2, signal2 = wavfile.read(file_path2)
sample_rate3, signal3 = wavfile.read(file_path3)

# Convert signals to mono if they are stereo
if len(signal1.shape) > 1:
    signal1 = signal1[:, 0]
if len(signal2.shape) > 1:
    signal2 = signal2[:, 0]
if len(signal3.shape) > 1:
    signal3 = signal3[:, 0]

# Normalize the signals
signal1 = signal1 / np.max(np.abs(signal1))
signal2 = signal2 / np.max(np.abs(signal2))
signal3 = signal3 / np.max(np.abs(signal3))

# Define parameters
frame_size = ?   # Set for typical value
hop_size = ?     # Set for typical value
num_mel_filters = 40
num_mfcc = 13

# Step 2: Calculate Mel filter bank
mel_points = np.linspace(0, 2595 * np.log10(1 + (sample_rate1 / 2) / 700), num_mel_filters + 2)
hz_points = 700 * (10**(mel_points / 2595) - 1)
bin_points = np.floor((frame_size + 1) * hz_points / sample_rate1).astype(int)
filters = np.zeros((num_mel_filters, int(np.floor(frame_size / 2 + 1))))
for m in range(1, num_mel_filters + 1):
    f_m_minus_1 = bin_points[m - 1]
    f_m = bin_points[m]
    f_m_plus_1 = bin_points[m + 1]
    for k in range(f_m_minus_1, f_m):
        filters[m - 1, k] = (k - bin_points[m - 1]) / (f_m - f_m_minus_1)
    for k in range(f_m, f_m_plus_1):
        filters[m - 1, k] = (bin_points[m + 1] - k) / (f_m_plus_1 - f_m)

# Function to calculate MFCCs without using custom functions
def calculate_mfcc(signal, sample_rate, frame_size, hop_size, filters, num_mfcc):
    num_frames = int(np.ceil(len(signal) / hop_size))
    padded_signal_length = num_frames * hop_size + frame_size
    padded_signal = np.zeros(padded_signal_length)
    padded_signal[:len(signal)] = signal
    stft_matrix = np.array([np.fft.rfft(padded_signal[i * hop_size:i * hop_size + frame_size])
                            for i in range(num_frames)])
    power_spectrum = np.abs(stft_matrix) ** 2
    mel_power = np.dot(power_spectrum, filters.T)
    log_mel_power = np.log(mel_power + 1e-12)
    mfccs = dct(log_mel_power, type=2, axis=1, norm='ortho')[:, :num_mfcc]
    return mfccs

# Calculate MFCCs and average them for each signal
mfccs1 = calculate_mfcc(signal1, sample_rate1, frame_size, hop_size, filters, num_mfcc)
mfccs2 = calculate_mfcc(signal2, sample_rate2, frame_size, hop_size, filters, num_mfcc)
mfccs3 = calculate_mfcc(signal3, sample_rate3, frame_size, hop_size, filters, num_mfcc)

# Average MFCCs over time (frames) for each sample
avg_mfccs1 = np.mean(mfccs1, axis=0)
avg_mfccs2 = np.mean(mfccs2, axis=0)
avg_mfccs3 = np.mean(mfccs3, axis=0)

# Step 5: Plot the average MFCCs for each sample
plt.figure(figsize=(15, 20))
plt.plot(avg_mfccs1, label=f"Sample 1 {file_path1}", marker='o')
plt.plot(avg_mfccs2, label=f"Sample 2 {file_path2}", marker='x')
plt.plot(avg_mfccs3, label=f"Sample 3 {file_path3}", marker='s')
plt.title("Average MFCCs Across Different Samples")
plt.xlabel("MFCC Coefficient Index")
plt.ylabel("Average MFCC Value")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()